## Fantasy NFL predictions for optimal roster by week
### This model will generate the optimal roster to start based on your own roster and the free agent pool. Currently only performs on QB, RB, WR, and TE as well as FLEX

## Model imports 
### Please make sure the following packages are installed (either with pip or conda)
#### numpy, pandas, warnings, nfl_data_py and catboost, espn_api, sleeperpy and yahoo_fantasy_api
#### Note - if you are running this on your local machine using Anaconda, please use conda instead of pip

In [ ]:
import numpy as np
import pandas as pd
%run fantasy_football_recommender.ipynb
import warnings
pd.options.mode.chained_assignment = None
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.set_option("display.max_columns",None)

#### Download all of the files needed from NFLFastR

In [ ]:
df_weekly_stats,df_schedules,df_draft_picks,df_rosters,df_injuries,df_contracts,df_ids = obtain_nfl_data()

#### Data cleaning for injury files

In [ ]:
df_injuries_use = clean_injuries(df_injuries)

#### Split players into categories - Currently only supports QB, RB, WR, and TE

In [ ]:
df_qbs,df_rbs,df_wrs,df_tes = split_players(df_weekly_stats)

#### Perform feature engineering for models

In [ ]:
df_qbs_fe = feature_engineering(df_qbs,df_rosters,df_injuries_use,df_schedules,df_contracts,"QB")
print("Done with qbs")
df_rbs_fe = feature_engineering(df_rbs,df_rosters,df_injuries_use,df_schedules,df_contracts,"RB")
print("Done with rbs")
df_wrs_fe = feature_engineering(df_wrs,df_rosters,df_injuries_use,df_schedules,df_contracts,"WR")
print("Done with wrs")
df_tes_fe = feature_engineering(df_tes,df_rosters,df_injuries_use,df_schedules,df_contracts,"TE")
print("Done with tes")

#### Read the model files and necessary features needed for model

In [ ]:
qb_features_df = pd.read_csv("qb_weekly_model_v2_hyperparameter_tuned.csv")
qb_model_path = "qb_weekly_model_v2_hyperparameter_tuned.cbm"
rb_features_df = pd.read_csv("rb_weekly_model_v2_hyperparameter_tuned.csv")
rb_model_path = "rb_weekly_model_v2_hyperparameter_tuned.cbm"
wr_features_df = pd.read_csv("wr_weekly_model_v2_hyperparameter_tuned.csv")
wr_model_path = "wr_weekly_model_v2_hyperparameter_tuned.cbm"
te_features_df = pd.read_csv("te_weekly_model_v2_hyperparameter_tuned.csv")
te_model_path = "te_weekly_model_v2_hyperparameter_tuned.cbm"

#### Make predictions for all players

In [ ]:
df_qb_preds = make_predictions(qb_model_path,qb_features_df,df_qbs_fe,"QB",df_rosters[df_rosters["season"]==2023])
df_rb_preds = make_predictions(rb_model_path,rb_features_df,df_rbs_fe,"RB",df_rosters[df_rosters["season"]==2023])
df_wr_preds = make_predictions(wr_model_path,wr_features_df,df_wrs_fe,"WR",df_rosters[df_rosters["season"]==2023])
df_te_preds = make_predictions(te_model_path,te_features_df,df_tes_fe,"TE",df_rosters[df_rosters["season"]==2023])

#### Get all available players on eihter your team or in free agency. Currently supported platforms: 
##### 1. ESPN
##### 2. Sleeper
##### 3. Yahoo 

In [ ]:
df_team,df_roster_max = determine_fantasy_engine(df_ids)

#### Currently not including kickers, bench, or defense

In [ ]:
df_roster_max = df_roster_max[(df_roster_max["position"]!="BE") & (df_roster_max["position"]!="D/ST") & (df_roster_max["position"]!="K")]

#### Determine the optimal roster based on points

In [ ]:
df_best_qb,df_best_rb,df_best_wr,df_best_te,df_best_flex = determine_optimal_roster(df_qb_preds,
                                                                                    df_rb_preds,df_wr_preds,
                                                                                    df_te_preds,df_team,df_roster_max)

#### Print out optimal roster from my prediction

In [ ]:
print_optimal_roster(df_best_qb)
print_optimal_roster(df_best_rb)
print_optimal_roster(df_best_wr)
print_optimal_roster(df_best_te)
print_optimal_roster(df_best_flex,flex=True)

#### Print out predictions for a single player

In [ ]:
df_use = df_te_preds
player_name = "Dalton Schultz"

In [ ]:
print_player_predictions(df_use,player_name)